In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
import re

In [5]:
data = pd.read_csv('Reviews.csv')

In [6]:
print(data.shape)

(568454, 10)


In [7]:
data.head()


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [8]:
data.isnull()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
568449,False,False,False,False,False,False,False,False,False,False
568450,False,False,False,False,False,False,False,False,False,False
568451,False,False,False,False,False,False,False,False,False,False
568452,False,False,False,False,False,False,False,False,False,False


In [9]:
data = data.dropna()

In [10]:
def get_wordlen(x):
    return len(x.split())
data['len'] = data.Text.apply(get_wordlen)
data = data[data.len<50]

In [11]:
data = data.sample(n=100000, random_state = 30)


In [12]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


In [13]:
def mod(x):
    if x < 3:
        return 0
    elif x >3 :
        return 1
    elif x == 3:
        return 2

data['Score'] = data['Score'].map(mod)
data['Score'].unique()

array([1, 0, 2], dtype=int64)

In [14]:
from bs4 import BeautifulSoup
import lxml
def preprocess_text(text):
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "can not", text)
    filtered_text = []
    for i in text.split():
        i = i.lower()
        if i not in stop_words:
            filtered_text.append(i)
    text = re.sub(r"http\S+", "", text)
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub("\S*\d\S*", "", text).strip()
    text = re.sub('[^A-Za-z]+', ' ', text)
    text = re.sub('<.*?>','',text)
    return text



In [15]:
text = data['Text'].values
preprocessed_text =[]
for i in text:
    preprocessed = preprocess_text(i)
    preprocessed_text.append(preprocessed)
    
preprocessed_text[:10]
    

['I fill this toy with cereal to keep my high maintenance Lab occupied for hours He is become very proficient at extracting pieces from the toy but it still is a challenge for him Fabulous indestructible toy Great investment for my money ',
 'Ahh the taste of cherries in the spring and the smell too When this arrived today I was eager to brew my first cup The contemplation this prompts in me is such a joy ',
 'If you like your Keurig you need to try this K cup Smooth taste with just a slight hint of flavor Not overwhelming like some brands My new favorite ',
 'The oz is the best way to get these and to save money They taste great I have already eaten half of thw Tub already',
 'We actually went to Big Bob is restaurant in Alabama and got hooked on his sauces We love the habanero red sauce because it has a little bit of a kick but not too much Try one and you will order a case just as we did ',
 'I do not have food allergies but am going to buy this mix A friend made it and it is ABSOLU

In [16]:
len(preprocessed_text)

100000

In [20]:
print(len(data['Score'].values))
print(len(preprocessed_text))
print(len(data['Text'].values))

100000
100000
100000


In [21]:
y = data["Score"].values
X = preprocessed_text
from sklearn.model_selection import train_test_split
X_train,x_test,y_train,y_test = train_test_split(X,y, test_size = 0.20, stratify = y, random_state = 33)
y[:100]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0,
       1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1], dtype=int64)

In [22]:
tf.keras.backend.clear_session()

max_seq_length = 50
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")

input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask")

segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")

bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

bert_model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=pooled_output)

In [23]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

In [46]:
print(type(do_lower_case))

<class 'numpy.bool_'>


In [24]:
import tokenization
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [25]:
def tokenization(X):
    max_seq_len = 50
    token_train = tokenizer.tokenize(X)
    token_train = token_train[0: (max_seq_len-2)]
    token_train = ['[CLS]', *token_train, '[SEP]']
    len_before = len(token_train)
    if (len(token_train) < max_seq_len):
        for i in range (len(token_train), 50):
            token_train.append('[PAD]')
    token_train = tokenizer.convert_tokens_to_ids(token_train)
    mask_train = ([1]*len_before + [0]* (max_seq_len - len_before))
    segment_train = [0]*max_seq_length
    
    return token_train, mask_train, segment_train





In [26]:
X_train_tokens = []
X_test_tokens = []
X_train_mask = []
X_test_mask = []
X_train_segment = []
X_test_segment = []

for i in X_train:
    token_train, mask_train, segment_train = tokenization(i)
    X_train_tokens.append(token_train)
    X_train_mask.append(mask_train)
    X_train_segment.append(segment_train)
    
for i in x_test:
    token_test, mask_test, segment_test = tokenization(i)
    X_test_tokens.append(token_test)
    X_test_mask.append(mask_test)
    X_test_segment.append(segment_test)

    
X_train_tokens = np.array(X_train_tokens)    
X_test_tokens = np.array(X_test_tokens)
X_train_mask = np.array(X_train_mask)
X_test_mask = np.array(X_test_mask)
X_train_segment = np.array(X_train_segment)
X_test_segment = np.array(X_test_segment)

In [27]:
print(X_train_tokens.shape)
print(X_train_mask.shape)
print(X_train_segment.shape)
print(X_test_tokens.shape)
print(X_test_mask.shape)
print(X_test_segment.shape)


(80000, 50)
(80000, 50)
(80000, 50)
(20000, 50)
(20000, 50)
(20000, 50)


In [28]:
print(type(X_train_tokens))

<class 'numpy.ndarray'>


In [ ]:
pickle.dump((X_train, X_train_tokens, X_train_mask, X_train_segment, y_train),open('train_data.pkl','wb'))
pickle.dump((x_test, X_test_tokens, X_test_mask, X_test_segment, y_test),open('test_data.pkl','wb'))

In [29]:
X_train_pooled_output=bert_model.predict([X_train_tokens,X_train_mask,X_train_segment])

In [30]:
X_test_pooled_output = bert_model.predict([X_test_tokens, X_test_mask, X_test_segment])

In [66]:
X_temp_exp = X_train_pooled_output[0].reshape(1,-1)
X_temp_exp.shape

(1, 768)

In [31]:
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, LSTM, Bidirectional
from tensorflow.keras.models import Model

In [32]:
tf.keras.backend.clear_session()

Input_layer = Input(shape = X_train_pooled_output.shape[1])
Layer1 = Dense(512, activation = 'relu', kernel_initializer = 'he_normal')(Input_layer)
Layer2 = Dense(256, activation = 'relu', kernel_initializer = 'he_normal')(Layer1)
Layer3 = Dense(128, activation = 'relu', kernel_initializer = 'he_normal')(Layer2)
Layer4 = Dense(64, activation ='relu', kernel_initializer = 'he_normal')(Layer2)
Output = Dense(3, activation = 'softmax', kernel_initializer = 'he_normal')(Layer3)

model_nn = Model(inputs = Input_layer, outputs = Output)
model_nn.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 768)]             0         
                                                                 
 dense (Dense)               (None, 512)               393728    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_4 (Dense)             (None, 3)                 387       
                                                                 
Total params: 558,339
Trainable params: 558,339
Non-trainable params: 0
_________________________________________________________________


In [33]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
y_train_new = encoder.fit_transform(y_train.reshape(-1,1)).toarray()
y_test_new = encoder.transform(y_test.reshape(-1,1)).toarray()
print(y_train_new.shape)
print(y_test_new.shape)

(80000, 3)
(20000, 3)


In [34]:
optimizer_nn = tf.keras.optimizers.Adam(learning_rate = 0.001)
tb_model_nn = tf.keras.callbacks.TensorBoard(log_dir = "logs_model_nn/", histogram_freq = 1)
model_nn.compile(optimizer = optimizer_nn, loss =tf.keras.losses.BinaryCrossentropy(), metrics =[tf.keras.metrics.AUC(from_logits = True), 'accuracy'])
history_nn = model_nn.fit(X_train_pooled_output, y_train_new, batch_size = 10, epochs = 10, validation_data = (X_test_pooled_output, y_test_new), callbacks =[tb_model_nn])


Epoch 1/10
8000/8000 [==============================] - 65s 8ms/step - loss: 0.2612 - auc: 0.9449 - accuracy: 0.8445 - val_loss: 0.3326 - val_auc: 0.9203 - val_accuracy: 0.7561
Epoch 2/10
8000/8000 [==============================] - 67s 8ms/step - loss: 0.2360 - auc: 0.9540 - accuracy: 0.8575 - val_loss: 0.2531 - val_auc: 0.9434 - val_accuracy: 0.8482
Epoch 3/10
8000/8000 [==============================] - 66s 8ms/step - loss: 0.2295 - auc: 0.9558 - accuracy: 0.8619 - val_loss: 0.2166 - val_auc: 0.9620 - val_accuracy: 0.8684
Epoch 4/10
8000/8000 [==============================] - 67s 8ms/step - loss: 0.2247 - auc: 0.9578 - accuracy: 0.8624 - val_loss: 0.2240 - val_auc: 0.9596 - val_accuracy: 0.8603
Epoch 5/10
8000/8000 [==============================] - 66s 8ms/step - loss: 0.2226 - auc: 0.9584 - accuracy: 0.8650 - val_loss: 0.2125 - val_auc: 0.9634 - val_accuracy: 0.8704
Epoch 6/10
8000/8000 [==============================] - 66s 8ms/step - loss: 0.2201 - auc: 0.9593 - accuracy: 0.866

In [69]:
model_nn.save("model_nn")
bert_model.save("model_bert")


INFO:tensorflow:Assets written to: model_nn\assets


INFO:tensorflow:Assets written to: model_nn\assets


INFO:tensorflow:Assets written to: model_bert\assets


INFO:tensorflow:Assets written to: model_bert\assets


In [74]:
from tensorflow.keras.models import load_model
model_nn_test = load_model("model_nn")
bert_model_test = load_model("model_bert")
bert_layer_test = hub.KerasLayer(hub.load("bert_layer_downloaded"))




In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

In [75]:
import tokenization
vocab_file_temp = bert_layer_test.resolved_object.vocab_file.asset_path.numpy()
do_lower_case_temp = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer_temp = tokenization.FullTokenizer(vocab_file_temp, do_lower_case_temp)




In [72]:
def tokenization_temp(X):
    max_seq_len = 50
    token_train = tokenizer_temp.tokenize(X)
    token_train = token_train[0: (max_seq_len-2)]
    token_train = ['[CLS]', *token_train, '[SEP]']
    len_before = len(token_train)
    if (len(token_train) < max_seq_len):
        for i in range (len(token_train), 50):
            token_train.append('[PAD]')
    token_train = tokenizer_temp.convert_tokens_to_ids(token_train)
    mask_train = ([1]*len_before + [0]* (max_seq_len - len_before))
    segment_train = [0]*max_seq_length
    
    return token_train, mask_train, segment_train

In [68]:
prediction_test = model_nn_test.predict(X_temp_exp)
prediction_test

array([[0.01513118, 0.9350167 , 0.04985212]], dtype=float32)

In [82]:
X_temp = 'I found this product to be extremely useful  to me. I would recommend anyone who is struggling with weight loss to use this product'
X_temp = preprocess_text(X_temp)
X_temp_tokens = []
X_temp_mask = []
X_temp_segment =[]
temp_tokens, temp_mask, temp_segment = tokenization_temp(X_temp)
X_temp_tokens.append(temp_tokens)
X_temp_mask.append(temp_mask)
X_temp_segment.append(temp_segment)
X_temp_tokens =np.array(X_temp_tokens)
X_temp_mask = np.array(X_temp_mask)
X_temp_segment = np.array(X_temp_mask)
X_temp_pooled_output = bert_model_test.predict([X_temp_tokens, X_temp_mask, X_temp_segment])
print(model_nn_test.predict(X_temp_pooled_output))


[[2.3279397e-05 9.9993253e-01 4.4232227e-05]]


In [80]:
X_temp_segment

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]